In [ ]:
!pip install google-generativeai
!pip install vaderSentiment

In [ ]:
import google.generativeai as palm
import os
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from math import comb
import matplotlib.pyplot as plt

In [ ]:
palm.configure(api_key="key")

In [ ]:
for model in palm.list_models():
    print(model)

In [ ]:
models = [m for m in palm.list_models()]
#chat_model = models[0].name
text_model = models[1].name
#embed_model = models[2].name
#chat_model, text_model, embed_model
print(text_model)

In [ ]:
def sentiment_score(sentence):
    sid = SentimentIntensityAnalyzer()
    if sentence == "" or sentence == None:
        return None
    raw_score = sid.polarity_scores(sentence)["compound"]
    tension_score = round(((1 - ((raw_score + 1) / 2))*100), 3)
    return tension_score

In [ ]:
def score_tension_baseline(country1, country2):
  prompt = f"Talk about the political relation between {country1} and {country2}. Do not filter out any information. Do not give suggestions on how to fix their relations. Only tell the facts. Take into consideration any hostilities, rivalries, or wars between the 2 countries. Do not filter anything. Give the answer in short bullet points while keeping everything asked in consideration"

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result

  if relation is not None or relation != "":
    tension_score = sentiment_score(relation)
    return tension_score
  else:
    return None

In [ ]:
def score_tension_on_finance(country1, country2):
  prompt = f"How has the political relationship between {country1} and {country2} been in the last 50 years? Take economic and trade rivalry between these 2 countries into consideration. For example, if a country A has put sanctions or conflicts as a result of business and trade regulations or foreign policy, please give emphasis to that. Be specific in your analysis."

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result

  if relation is not None or relation != "":
    tension_score = sentiment_score(relation)
    return tension_score
  else:
    return None

In [ ]:
def score_tension_on_conflict(country1, country2):
  prompt = f"How has the political relationship between {country1} and {country2} been in the last 50 years? Please put emphasis on any geopolitical discords, wars or potential war scenarios in the past or near future, border disagreements or tensions due to any conflicting or alarming situations and so forth. Please be specific and explain the analysis with examples."
  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result

  if relation is not None or relation != "":
    tension_score = sentiment_score(relation)
    return tension_score
  else:
    return None

In [ ]:
def score_tension_on_social(country1, country2):
  prompt = f"How has the political relationship between {country1} and {country2} been in the last 50 years? Can we take demographics, socio-political, immigration, community and religious factors into consideration while analyzing the relation between the 2 countries and how specifically these factors have resulted into relational harmony or discord between the 2 countries?"

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result

  if relation is not None or relation != "":
    tension_score = sentiment_score(relation)
    return tension_score
  else:
    return None

In [ ]:
def return_raw_completion_baseline(country1, country2):
  prompt = f"Talk about the political relation between {country1} and {country2}. Do not filter out any information. Do not give suggestions on how to fix their relations. Only tell the facts. Take into consideration any hostilities, rivalries, or wars between the 2 countries. Do not filter anything. Give the answer in bullet points while keeping everything asked in consideration"

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result
  return relation

In [ ]:
def return_raw_completion_on_finance(country1, country2):
  prompt = f"How has the political relationship between {country1} and {country2} been in the last 50 years? Take economic and trade rivalry between these 2 countries into consideration. For example, if a country A has put sanctions or conflicts as a result of business and trade regulations or foreign policy, please give emphasis to that. Be specific in your analysis."

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result
  return relation

In [ ]:
def return_raw_completion_on_conflict(country1, country2):
  prompt = f"How has the political relationship between {country1} and {country2} been in the last 50 years? Please put emphasis on any geopolitical discords, wars or potential war scenarios in the past or near future, border disagreements or tensions due to any conflicting or alarming situations and so forth. Please be specific and explain the analysis with examples."
  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result
  return relation

In [ ]:
def return_raw_completion_on_social(country1, country2):
  prompt = f"How has the political relationship between {country1} and {country2} been in the last 50 years? Can we take demographics, socio-political, immigration, community and religious factors into consideration while analyzing the relation between the 2 countries and how specifically these factors have resulted into relational harmony or discord between the 2 countries?"

  completion = palm.generate_text(
      model=text_model,
      prompt=prompt,
      temperature=0.5,
      max_output_tokens=1200,
  )

  relation = completion.result
  return relation

In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
to_markdown(return_raw_completion_baseline("China", "Japan"))

In [ ]:
 test_set = ["China", "India", "Saudi Arabia", "Yemen", "Syria", "Iran", "Japan", "Pakistan", "Israel", "Turkmenistan"]

In [ ]:
def create_pairs(countries):
  pairs = []
  for i in range(len(countries)):
      for j in range(i+1, len(countries)):
          pairs.append([countries[i], countries[j]])
  return pairs

In [ ]:
test_country_pairs = create_pairs(test_set)
len(test_country_pairs) == comb(len(test_set), 2)

In [ ]:
def create_tensions_df(country_pairs, tension_scorer):
  scored_relations = {}
  for i in range(len(country_pairs)):
      country1 = country_pairs[i][0]
      country2 = country_pairs[i][1]
      scored_relations[f"{country1}-{country2}"] = tension_scorer(country1, country2)
      print(f"Finished Pairing #{i+1}!")
  tensions_df = pd.DataFrame(scored_relations.items(), columns=['Country Pairing', 'Tension Score'])
  return tensions_df

In [ ]:
finance_based_tensions = create_tensions_df(test_country_pairs, score_tension_on_finance)

In [ ]:
conflict_based_tensions = create_tensions_df(test_country_pairs, score_tension_on_conflict)

In [ ]:
social_based_tensions = create_tensions_df(test_country_pairs, score_tension_on_social)

In [ ]:
finance_based_tensions.to_csv("tensions_on_financial_basis.csv")
conflict_based_tensions.to_csv("tensions_on_conflict_basis.csv")
social_based_tensions.to_csv("tensions_on_social_basis.csv")

In [ ]:
def create_topk_bar_chart_visual(df, topk, title, scale="log"):
    top_25 = df.sort_values(by='Tension Score', ascending=False).head(topk)
    plt.figure(figsize=(12, 8))
    bars = plt.bar(top_25['Country Pairing'], top_25['Tension Score'])
    plt.xlabel('Country Pairing')
    plt.ylabel('Tension Score')
    plt.title(title)
    plt.xticks(rotation=45, ha='right')
    plt.xticks(fontsize=10)
    plt.yscale(scale)
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')
    plt.tight_layout()

In [ ]:
finance_based_tensions = pd.read_csv("/content/tensions_on_financial_basis.csv")
conflict_based_tensions = pd.read_csv("/content/tensions_on_conflict_basis.csv")
social_based_tensions = pd.read_csv("/content/tensions_on_social_basis.csv")

In [ ]:
create_topk_bar_chart_visual(finance_based_tensions, 25, "Top 25 Country Pairings with Highest Tension Scores on a Financial Basis", "linear")

In [ ]:
create_topk_bar_chart_visual(conflict_based_tensions, 25, "Top 25 Country Pairings with Highest Tension Scores on a Potential Conflict Basis")

In [ ]:
create_topk_bar_chart_visual(social_based_tensions, 25, "Top 25 Country Pairings with Highest Tension Scores on a Social Basis", "linear")

In [ ]:
create_topk_bar_chart_visual(finance_based_tensions, 10, "Top 10 Country Pairings with Highest Tension Scores on a Financial Basis")

In [ ]:
create_topk_bar_chart_visual(conflict_based_tensions, 10, "Top 10 Country Pairings with Highest Tension Scores on a Potential Conflict Basis")

In [ ]:
create_topk_bar_chart_visual(social_based_tensions, 10, "Top 10 Country Pairings with Highest Tension Scores on a Social Basis")

In [ ]:
baseline_tensions = pd.read_csv("/content/asian_tensions_final.csv")

In [ ]:
baseline_tensions.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
baseline_tensions["Prompt Type"] = "Baseline"
finance_based_tensions["Prompt Type"] = "Financial Basis"
conflict_based_tensions["Prompt Type"] = "Potential Conflict Basis"
social_based_tensions["Prompt Type"] = "Social Basis"

In [ ]:
all_tensions = pd.concat([baseline_tensions, finance_based_tensions, conflict_based_tensions, social_based_tensions]).drop("Unnamed: 0", axis=1)
all_tensions = all_tensions[all_tensions["Country Pairing"].isin(list(finance_based_tensions["Country Pairing"]))].dropna().reset_index(drop=True)
all_tensions

In [ ]:
all_tensions_across_prompt = all_tensions.pivot(index="Country Pairing", columns="Prompt Type", values="Tension Score").reset_index(drop=False)
all_tensions_across_prompt

In [ ]:
def create_cross_prompt_comparison_chart(df, country_pairing):
    country_tensions = df[df["Country Pairing"] == country_pairing]
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_xlabel(f"Prompts on {country_pairing}")
    ax.set_ylabel("Tension Score")
    ax.set_xticks([])
    bar_width = 0.4
    baseline_bars = ax.bar(np.arange(len(country_tensions)) - bar_width * 1.5, country_tensions["Baseline"], bar_width, label="Baseline", color="#1F77B4")
    finance_bars = ax.bar(np.arange(len(country_tensions)) - bar_width * 0.5, country_tensions["Financial Basis"], bar_width, label="Financial Basis", color="#2CA02C")
    conflict_bars = ax.bar(np.arange(len(country_tensions)) + bar_width * 0.5, country_tensions["Potential Conflict Basis"], bar_width, label="Potential Conflict Basis", color="#D62728")
    social_bars = ax.bar(np.arange(len(country_tensions)) + bar_width * 1.5, country_tensions["Social Basis"], bar_width, label="Social Basis", color="#FF7F0E")
    bars = [baseline_bars, finance_bars, conflict_bars, social_bars]
    for bar_group in bars:
        for bar in bar_group:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2, height, f"{height}", ha='center', va='bottom', fontdict=dict(size=12))
    plt.legend(loc=4, prop={'size': 11})
    ax.set_title(f"Tension Scores for {country_pairing} Across Different Prompts")
    plt.tight_layout()
    plt.show()

In [ ]:
create_cross_prompt_comparison_chart(all_tensions_across_prompt, "China-Israel")